In [32]:
import numpy as np
import pandas as pd
TRAIN_CSV_NAME = 'train_data.csv'
TEST_CSV_NAME = 'test_data.csv'
TRAIN_IMG_DIRECTORY = 'train_posters'
TEST_IMG_DIRECTORY = 'test_posters'
SEED = 1
np.random.seed(SEED) # Seed our randomness for reproducibilty

# Choose indices for our test data
testIdx = np.random.choice(y.shape[0], 200, replace=False)

#First load and preprocess the TFIDF model.

In [9]:
from bg.build_tfidf import preprocess_nonimage_data
Xtfidf, Xother, y = preprocess_nonimage_data(TRAIN_CSV_NAME)
X = np.zeros([Xtfidf.shape[0], Xtfidf.shape[1] + 2])
X[:,0:2], X[:, 2:]  = Xother[:, 2:], Xtfidf

X_test = X[testIdx, :]
X_train = np.delete(X, testIdx, axis=0)
y_test = y[testIdx]
y_train = np.delete(y, testIdx)



Now let's train an SVM on the TFIDF data vectors.

In [10]:
from sklearn import svm
clf = svm.LinearSVC(penalty='l2', dual=True, multi_class='ovr', max_iter=100000)
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=100000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [13]:
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(X_test)
a = confusion_matrix(y_test, y_pred)
print("Classification rate %f" %(a.trace() / a.sum()))

Classification rate 0.590000


#Now load and preprocess the color histogram model.

In [24]:
from bg.util import build_histograms, preprocess_images
from sklearn.ensemble import RandomForestClassifier
train_data_csv = pd.read_csv(TRAIN_CSV_NAME).values
genres = np.array(train_data_csv[:,-1]).astype(int)
test_data_csv = pd.read_csv(TEST_CSV_NAME).values
preprocessed_train = preprocess_images(train_data_csv, TRAIN_IMG_DIRECTORY)
histogram_processed_train = build_histograms(preprocessed_train, 12)

bgX_test = histogram_processed_train[testIdx]
bgX_train = np.delete(histogram_processed_train, testIdx, axis=0)
bgy_test = genres[testIdx]
bgy_train = np.delete(genres, testIdx)



/home/ben/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


And now fit/test it

In [35]:
bg_model = RandomForestClassifier(n_estimators=800, random_state=SEED)
bg_model.fit(bgX_train, bgy_train)
print(bg_model.score(bgX_test, bgy_test))
print(bg_model.score(bgX_test, y_test)) # This proves that we are using the same labels

0.475
0.475


# Finally let's try an ensemble model

In [36]:
from sklearn.svm import SVC
from bg.util import trainModelStack, predictModelStack
combiner = SVC() # Standard support vector classifier with RBF kernel
models = [bg_model, clf]
datas = [bgX_train, X_train]
trainModelStack(models, combiner, datas, y_train)
predictions = predictModelStack(models, combiner, [bgX_test, X_test])


AttributeError: 'LinearSVC' object has no attribute 'predict_proba'